In [1]:
import os
import sys
import pyspark
import pandas as pd
from functools import reduce
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.functions import max as _max, min as _min, col
from pyspark.sql.functions import *
import numpy as np
from IPython.core.display import HTML
from sklearn.metrics.pairwise import cosine_similarity
from pyspark.sql.types import *
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.feature import Normalizer
from pyspark.mllib.linalg.distributed import IndexedRow, IndexedRowMatrix

In [2]:
conf = pyspark.SparkConf().setAll([('spark.executor.memory', '10g'),  # ~45g
                                   ('spark.dynamicAllocation.enabled','true'),
                                   ('spark.shuffle.service.enabled', 'true'),
                                   ('spark.authenticate', 'true'),
                                   ('spark.dynamicAllocation.initialExecutors', '20'),
                                   ('spark.dynamicAllocation.minExecutors', '4'),
                                   ('spark.dynamicAllocation.maxExecutors', '20'),
                                   ('spark.dynamicAllocation.executorIdleTimeout', '2000'),
                                   ('spark.dynamicAllocation.cachedExecutorIdleTimeout', '21600'),
                                   ('spark.driver.cores','2'),   # ~4
                                   ('spark.executor.cores', '4'),  # ~40 - max
                                   ("spark.driver.maxResultSize", "4g"),
                                   ('spark.yarn.queue', 'araadh_q1.uhc_oa_pi_dev_sq1'),
                                   ('spark.driver.memory','10g'),
                                   ("spark.sql.execution.arrow.pyspark.enabled", "true")])
spark = SparkSession.builder.appName("Assignment").config(conf=conf).enableHiveSupport().getOrCreate().newSession()
spark

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [4]:
df = spark.read.format("csv").option("header", "true").option('inferSchema', True).load("/home/parora41/Masters/train.csv")
df.show(2)

+----------+--------------------+------------+----------------+----------------+----------+-----------+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+------+--------------+-----------+
|Unnamed: 0|            track_id|     artists|      album_name|      track_name|popularity|duration_ms|explicit|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence| tempo|time_signature|track_genre|
+----------+--------------------+------------+----------------+----------------+----------+-----------+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+------+--------------+-----------+
|         0|5SuOikwiRyPMVoIQD...| Gen Hoshino|          Comedy|          Comedy|        73|     230666|   False|       0.676| 0.461|  1|  -6.746|   0|      0.143|      0.0322|         1.01E-6|   0.358|  0.715|87.917|           4.0|   acoustic|
|         1|4qPNDBW1i3p1

In [5]:
df.printSchema()

root
 |-- Unnamed: 0: integer (nullable = true)
 |-- track_id: string (nullable = true)
 |-- artists: string (nullable = true)
 |-- album_name: string (nullable = true)
 |-- track_name: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- duration_ms: string (nullable = true)
 |-- explicit: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- key: string (nullable = true)
 |-- loudness: string (nullable = true)
 |-- mode: string (nullable = true)
 |-- speechiness: string (nullable = true)
 |-- acousticness: string (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- liveness: string (nullable = true)
 |-- valence: string (nullable = true)
 |-- tempo: double (nullable = true)
 |-- time_signature: double (nullable = true)
 |-- track_genre: string (nullable = true)



In [6]:
# Shape of DataFrame
print("Rows : {}\nColumns : {}".format(df.count(), len(df.columns)))

Rows : 114000
Columns : 21


In [7]:
## checking NULLS
null_df = df.select([count(when(col(c).contains('None') |
                               col(c).contains("null") |
                               (col(c) == "NaN") |
                                col(c).isNull() |
                               isnan(c), c)).alias(c) for c in df.columns])
null_df.show()

+----------+--------+-------+----------+----------+----------+-----------+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-----+--------------+-----------+
|Unnamed: 0|track_id|artists|album_name|track_name|popularity|duration_ms|explicit|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|tempo|time_signature|track_genre|
+----------+--------+-------+----------+----------+----------+-----------+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-----+--------------+-----------+
|         0|       0|      4|        10|        11|         0|          0|       0|           0|     0|  0|       0|   0|          0|           0|               0|       0|      0|    0|             0|          0|
+----------+--------+-------+----------+----------+----------+-----------+--------+------------+------+---+--------+----+-----------+-----------

In [8]:
## Removing nulls from the df
df1 = df.filter((df['artists'] != 'null')&(df['album_name'] != 'null')&(df['track_name'] != 'null'))

In [9]:
df1.createOrReplaceTempView("df1")

### Top 5 genre which are most popluar among users

In [10]:
spark.sql(""" select track_genre, sum(popularity) `popularity` from df1 group by track_genre order by popularity desc """).show(5)

+-----------+----------+
|track_genre|popularity|
+-----------+----------+
|   pop-film|   59169.0|
|      k-pop|   56896.0|
|      chill|   53651.0|
|        sad|   52379.0|
|     grunge|   49594.0|
+-----------+----------+
only showing top 5 rows



### Summary of the top 5 tracks

In [11]:
# casting the numerical columns for proper feature creation
sdf1 = spark.sql(""" select distinct track_id,
cast(popularity as int) `popularity`,
cast(duration_ms as int) `duration_ms`,
cast(danceability as float) `danceability`,
cast(energy as float) `energy`,
cast(loudness as float) `loudness`,
cast(mode as int) `mode`,
cast(speechiness as float) `speechiness`,
cast(acousticness as float) `acousticness`,
cast(instrumentalness as float) `instrumentalness`,
cast(liveness as float) `liveness`,
cast(valence as float) `valence`,
cast(tempo as float) `tempo`,
cast(time_signature as int) `time_signature`
from df1 """)
sdf1.createOrReplaceTempView("sdf1")

In [12]:
sdf1.summary().show()

+-------+--------------------+------------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+
|summary|            track_id|        popularity|       duration_ms|      danceability|           energy|          loudness|             mode|       speechiness|      acousticness|  instrumentalness|          liveness|            valence|             tempo|    time_signature|
+-------+--------------------+------------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+
|  count|               90460|             90340|             90425|             90366|            90428|             90455|            90454|             90456|        

In [13]:
spark.sql(""" select track_name, max(popularity) `popularity`, avg(danceability) `danceability`,
avg(energy) `energy`,
avg(loudness) `loudness`,
avg(mode) `mode`,
avg(speechiness) `speechiness`,
avg(acousticness) `acousticness`,
avg(instrumentalness) `instrumentalness`,
avg(liveness) `liveness`,
avg(valence) `valence`,
avg(tempo) `tempo`
from df1 
where trim(popularity) between 0 and 100
group by track_name order by popularity desc """).show(5, truncate=False)

+-------------------------------------+----------+------------+------------------+--------+----+-----------+------------+----------------+--------+-------------------+-------+
|track_name                           |popularity|danceability|energy            |loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence            |tempo  |
+-------------------------------------+----------+------------+------------------+--------+----+-----------+------------+----------------+--------+-------------------+-------+
|Quevedo: Bzrp Music Sessions, Vol. 52|99        |0.621       |0.782             |-5.548  |1.0 |0.044      |0.0125      |0.033           |0.23    |0.55               |128.033|
|I'm Good (Blue)                      |98        |0.561       |0.9650000000000001|-3.673  |0.0 |0.0343     |0.00383     |7.07E-6         |0.371   |0.30400000000000005|128.04 |
|La Bachata                           |98        |0.835       |0.679             |-5.329  |0.0 |0.0364     |0.583       

### Getting most similar tracks for recommendation using features similarity

In [14]:
# Creating features out of appropriate columns and normalizing them
vectorAssembler = VectorAssembler(inputCols = ['popularity', 'danceability', 'energy', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo'],
                                  outputCol = 'features')
df2 = vectorAssembler.setHandleInvalid("skip").transform(sdf1)
normalizer = Normalizer(inputCol="features", outputCol="norm")
df3 = normalizer.transform(df2)

In [15]:
df3.show(2)

+--------------------+----------+-----------+------------+------+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+--------------------+--------------------+
|            track_id|popularity|duration_ms|danceability|energy|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|time_signature|            features|                norm|
+--------------------+----------+-----------+------------+------+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+--------------------+--------------------+
|0ltIx8PLjtWoS9zn9...|        51|     322146|       0.592| 0.769|  -5.268|   1|     0.0311|       0.319|         4.66E-6|   0.137|  0.385|102.033|             4|[51.0,0.592000007...|[0.44658398202370...|
|68HbD7rEFTlnJy4UW...|        28|     217466|       0.578| 0.636|  -5.506|   1|     0.0342|      0.0874|             0.0|   0.124|  0.556|  124.9|             4|[28.0,0.578000009...|[0

In [16]:
# crearting "ID" column for processing
df3.createOrReplaceTempView("df3")
df4 = spark.sql(""" select *, row_number() OVER(order by track_id) `ID` from df3 """)
df4.createOrReplaceTempView("df4")

In [17]:
df5 = spark.sql(""" select * from df4 limit 500 """) # test sample
mat = IndexedRowMatrix(
    df5.select("ID", "norm")\
        .rdd.map(lambda row: IndexedRow(row.ID, row.norm.toArray()))).toBlockMatrix()
dot = mat.multiply(mat.transpose())
array_values = dot.toLocalMatrix().toArray()

In [18]:
# sample similarity matrix
pdf = pd.DataFrame(array_values)
pdf.tail(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500
499,0.0,0.948463,0.895373,0.881371,0.923605,0.864495,0.849305,0.776416,0.870619,0.933894,0.836160,0.827627,0.825612,0.956285,0.944794,0.947146,0.844373,0.878069,0.856424,0.857589,0.94904,0.879883,0.917004,0.819931,0.923003,0.926821,0.961061,0.926408,0.953195,0.885807,0.944969,0.943090,0.801398,0.803968,0.828877,0.890163,0.955487,0.844879,0.868807,0.853443,0.791349,0.943923,0.891422,0.897101,0.912741,0.953371,0.933881,0.936089,0.902876,0.948900,0.890379,0.784033,0.922708,0.972001,0.935034,0.820528,0.964590,0.889193,0.770136,0.849771,0.771129,0.941955,0.857909,0.899548,0.897778,0.974283,0.963642,0.809646,0.92017,0.921023,0.937432,0.879050,0.898684,0.783996,0.851100,0.998294,0.903768,0.782581,0.959343,0.873680,0.924701,0.872105,0.924585,0.786315,0.831115,0.972136,0.993889,0.939061,0.877684,0.934837,0.865325,0.842222,0.914939,0.868559,0.858171,0.912628,0.834916,0.913329,0.910108,0.780438,0.86605,0.944329,0.851518,0.788560,0.876064,0.875757,0.966233,0.816794,0.874891,0.877382,0.912661,0.914586,0.777643,0.781168,0.949855,0.841850,0.912164,0.840947,0.777189,0.866214,0.927140,0.818931,0.894390,0.88958,0.848658,0.935050,0.773790,0.775975,0.915045,0.90165,0.892884,0.947711,0.844849,0.954649,0.867379,0.830611,0.969786,0.903144,0.894425,0.851119,0.87864,0.969740,0.873832,0.820970,0.781254,0.865824,0.958968,0.830956,0.880588,0.943794,0.927760,0.855633,0.832673,0.863205,0.877747,0.912868,0.907812,0.775210,0.835298,0.975767,0.776360,0.805029,0.959134,0.782774,0.934627,0.999004,0.910652,0.897470,0.868097,0.944034,0.900293,0.849033,0.780751,0.928345,0.924951,0.950711,0.862501,0.947448,0.964793,0.936067,0.952352,0.825062,0.778953,0.984050,0.932951,0.93234,0.962982,0.792711,0.884671,0.968309,0.801802,0.785114,0.807724,0.777021,0.884246,0.939638,0.892213,0.952980,0.780967,0.818243,0.834004,0.960544,0.874610,0.937150,0.898543,0.844346,0.849226,0.951760,0.897196,0.854118,0.945835,0.881137,0.777730,0.867884,0.792027,0.838607,0.792863,0.959838,0.949299,0.906027,0.816591,0.959040,0.920931,0.936963,0.866927,0.812331,0.871431,0.812425,0.798463,0.908388,0.966366,0.953285,0.834314,0.820649,0.90435

In [19]:
# df.dropna(inplace=True)
# df.drop_duplicates(subset=['track_name'],inplace=True,keep='first',ignore_index=True)
# df1 = df[['popularity', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]
# df2 = pd.concat([df['track_id'], df1], axis=1)
# df2.head(2)
# df3 = pd.DataFrame(cosine_similarity(df1.head(500)))
# df3.columns = df2['track_id'].head(500)
# df4 = df3.round(3)

# df4 = df4.replace(1.000, 0.0)
# df4['Max'] = df4.idxmax(axis=1)
# df6 = pd.concat([df2[['track_id']].head(500), df4], axis=1)
# df6[['track_id', 'Max']].to_csv('/home/parora41/bds_sample.csv')

In [20]:
spark.stop()